In [ ]:
import requests
from langchain.schema import Document
from bs4 import BeautifulSoup

api_key = "up_qAeV4wltdGnqdmxavmqmxXyRBpmcZ" # ex: up_xxxYYYzzzAAAbbbCCC
file_path = "data/sample_pdf.pdf" # ex: ./image.png

response = requests.post(
    "https://api.upstage.ai/v1/document-digitization",
    headers={"Authorization": f"Bearer {api_key}"},
    files={"document": open(file_path, "rb")},
    data={
      "ocr": "force", # OCR을 강제로 수행하도록 설정 ("auto"로 설정 시 이미지 문서에서만 OCR 수행)
      "coordinates": False, # 각 레이아웃 요소의 위치 정보 반환 여부
      "chart_recognition": True, # 차트 인식 여부 
      "output_formats": '["html"]', # 결과를 HTML 형식으로 반환 (text, markdown도 가능)
      "base64_encoding": '["table"]', # 표에 대한 base64 인코딩 요청
      "model": "document-parse" # 사용할 모델 지정 
    }
    )


print(response.json())

{'api': '2.0', 'content': {'html': '<table id=\'0\' style=\'font-size:16px\'><thead></thead><tbody><tr><td>1. 정책/법제</td><td>2. 기업/산업</td><td>3. 기술/연구</td><td>4. 인력/교육</td></tr></tbody></table>\n<p id=\'1\' data-category=\'paragraph\' style=\'font-size:22px\'>갈릴레오의 LLM 환각 지수 평가에서 GPT-4가 가장 우수</p>\n<p id=\'2\' data-category=\'paragraph\' style=\'font-size:18px\'>KEY Contents</p>\n<p id=\'3\' data-category=\'list\' style=\'font-size:20px\'>주요 LLM의 환각 현상을 평가한 \'LLM 환각 지수\'에 따르면 GPT-4는 작업 유형과 관계없이<br>가장 우수한 성능을 보였으며 GPT-3.5도 거의 동등한 성능을 발휘</p>\n<br><p id=\'4\' data-category=\'list\' style=\'font-size:20px\'>오픈소스 모델 중에서는 메타의 라마2가 RAG 없는 질문과 답변 및 긴 형식의 텍스트<br>생성에서 가장 우수한 성능을 발휘</p>\n<p id=\'5\' data-category=\'paragraph\' style=\'font-size:20px\'>주요 LLM 중 GPT-4가 가장 환각 현상 적고 GPT-3.5 터보도 비슷한 성능 기록</p>\n<p id=\'6\' data-category=\'list\' style=\'font-size:18px\'>■ 머신러닝 데이터 관리 기업 갈릴레오(Galileo)가 2023년 11월 15일 주요 LLM의 환각 현상을 평가한<br>\'LLM 환각 지수(LLM Hallucination Index)\'를 발표</p>\n<br><p id=\'7\' data

In [2]:
datas = response.json()
# temp["elements"][0]

In [10]:
datas = response.json()["elements"]

In [11]:
# html = str(datas['elements'][3]['content']['html'])
# html

In [12]:
from openai import OpenAI # openai==1.52.2

client = OpenAI(
    api_key=api_key,
    base_url="https://api.upstage.ai/v1"
)

# pdf에 도표나 공식이 많을경우, 비동기 처리기법을 통해서 처리 속도를 늘리는 개선방안이 필요하다.
def content_convert(prompt, data):
    massage = prompt + data

    stream = client.chat.completions.create(
        model="solar-pro",
        messages=[
            {
                "role": "user",
                "content": massage
            }
        ],
        stream=True,
    )

    result = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            result += chunk.choices[0].delta.content
    return result

print(content_convert("나는 7살짜리 어린아이야. ", "달은 지구랑 얼마나 멀어?"))


달은 지구에서 약 238,855마일(384,400킬로미터) 떨어져 있어요. 이는 지구의 둘레보다 약 30배 정도 더 먼 거리랍니다! 그래서 달이 하늘에서 꽤 크게 보이는데도 불구하고, 그렇게 멀리 떨어져 있는 거죠.


In [13]:
'''
to do :
equation_handler, chart_handler, figure_handler 에 content_converter를 적용해서 
converted_text = " 제공된 데이터 테이블에는 여러 모델과 item_01이라는 지표에 대한 성능이 나열되어 있습니다. ... mpT-7b-instruct 모델은 0.58의 점수로 테이블에서 가장 낮은 점수를 기록했습니다."
형태의 output이 나오게 해야함.

equation_handler, chart_handler, figure_handler 에 각각 맞는 프롬포트를 성정해야함

++
시간이 꽤나 걸릴수 있음으로, 가능하면 비동기식 처리방법을 채택하는것도 고려해봐야함.
'''

'\nto do :\nequation_handler, chart_handler, figure_handler 에 content_converter를 적용해서 \nconverted_text = " 제공된 데이터 테이블에는 여러 모델과 item_01이라는 지표에 대한 성능이 나열되어 있습니다. ... mpT-7b-instruct 모델은 0.58의 점수로 테이블에서 가장 낮은 점수를 기록했습니다."\n형태의 output이 나오게 해야함.\n\nequation_handler, chart_handler, figure_handler 에 각각 맞는 프롬포트를 성정해야함\n\n++\n시간이 꽤나 걸릴수 있음으로, 가능하면 비동기식 처리방법을 채택하는것도 고려해봐야함.\n'

In [14]:
# BeautifulSoup을 이용항 파싱
# 모든 데이터는 결국 embedding vector 적용이 가능한 시퀀셜한 텍스트로 변환돼야한다. 
'''
추상함수  category_handler()
category_handler를 상속할 함수들  = [equation_handler, chart_handler, figure_handler]
param : 
html || markdown (str)
return : sequantial natural language (str)

에시:
chart_html =  "<figure id='4' data-category='chart'><img data-coord=\"top-left:(175,1305); bottom-right:(785,1784)\" /><figcaption><p>Chart Type: bar</p></figcaption><table><t...34%</td></tr></table></figure>",
converted_text = category_handler(chart_html)
converted_text = " 제공된 데이터 테이블에는 여러 모델과 item_01이라는 지표에 대한 성능이 나열되어 있습니다. ... mpT-7b-instruct 모델은 0.58의 점수로 테이블에서 가장 낮은 점수를 기록했습니다."
'''
from bs4 import BeautifulSoup

# 각각의 헨들러에 맞게 llm이 텍스트를 생성하도록, 프롬포트 엔지니어링용 프롬포트가 필요해. 모든 llm의 출력결과는 임베딩 벡터를 뽑을수 있는 sequantial 한 텍스트 형태여야해.
# llm에 prompt + question 형태로 원하는 입력값을 넣어서, 원하는 output 형식으로 답이 나오게 만들꺼야. 아랜 question, output 예시를 줬으니, 너는 각 목적에 맞는 prompt를 작성해줘.



def text_handler(figure_data: dict | None) -> str:
    """
    어떤 HTML 데이터든 시퀀셜한 자연어 텍스트로 반환.
    None, 형식 불일치, HTML 파싱 실패 등도 모두 "" 또는 안전한 문자열 반환.
    """
    if figure_data is None or not isinstance(figure_data, dict):
        return ""

    try:
        html = figure_data.get("content", {}).get("html", "")
        if not html:
            return ""

        soup = BeautifulSoup(html, "html.parser")

        img = soup.find("img")
        if img and img.get("alt"):
            alt_text = img.get("alt").strip()
            return f"markdown: {alt_text}, text: {alt_text}"

        text = soup.get_text(separator="\n", strip=True)
        return text
    except Exception as e:
        # 에러가 발생해도 빈 문자열 반환
        return ""

def equation_handler(equation_data_raw: tuple):
    equation_handler_prompt = (
        "너는 수학 수식을 설명하는 한국어 선생님이야.\n"
        "아래에 주어진 수학 수식의 LaTeX 마크다운(markdown) 표현과 일반 텍스트(text) 표현을 보고, "
        "이 수식이 의미하는 수학적 개념 또는 연산 과정을 한국어 자연어 문장으로 간결하게 설명해줘.\n"
        "결과 문장은 논리적 흐름을 가지는 완전한 문장이어야 하며, 수학적 표현이 자연스럽게 해석되도록 구성되어야 해.\n"
        "수식의 구조적 의미에 초점을 맞춰 설명해줘.\n"
        "형식 예시: 이 수식은 k=1부터 n까지의 합을 나타내며, 분자는 2k+1이고 분모는 1^2부터 k^2까지의 제곱합이다."
    )
    # 실제 방정식 정보를 지닌 데이터.
    equation_data = equation_data_raw[0]
    # 방성식 정보위에 있는 문단의 데이터. (방정식에 대한 설명을 하고있을 확률 높음.) 해당 데이터는 None 이 들어올수있음. 그래도 오류가 나지않게 예외처리 해야함.
    equation_descript_text = text_handler(equation_data_raw[1])
    
    # 문자열 파싱을 통한 텍스트 추출
    markdown = equation_data["content"].get("markdown", "").strip()
    text = equation_data["content"].get("text", "").strip()
    equation_text =  f"markdown: {markdown}, text: {text}"
    print(equation_descript_text)
    # llm에 context와 함께 제공해서, 시퀀셜한 자연어 문장으로 결과값 받기.
    llm_input = equation_descript_text + " the equation is : " +equation_text
    result = content_convert(prompt=equation_handler_prompt, data=llm_input)
    return result


def chart_handler(chart_data_raw: tuple):
    # chart_handler
    chart_handler_prompt = (
        "너는 표에 포함된 수치를 한국어로 요약 설명하는 AI야.\n"
        "아래에 제공된 표 내용(markdown, text)을 보고, 각 항목과 수치를 한국어 문장으로 자연스럽게 설명해줘.\n"
        "표는 항목별 수치를 나타내는 도표이며, 각 항목의 이름과 그에 대응하는 수치를 연결하여 설명해야 해.\n"
        "결과는 문장 형태의 한국어 설명이어야 하며, 사람이 쉽게 이해할 수 있는 시퀀셜한 설명으로 구성돼야 해.\n"
        "형식 예시: 이 도표는 에너지 생산량 비중을 나타내며, 핵에너지는 4%, 재생에너지는 4%, 석유는 34%의 비중을 차지한다."
    )

    # 실제 데이터 도표 html 정보.
    chart_data = chart_data_raw[0]["content"]["html"]

    # 도표에 대한 설명이 담긴 텍스트 (도표 윗문장으로, 도표에 대한 설명이 있을 확률이 높다.)
    chart_descript_text = text_handler(chart_data_raw[1])
    print(chart_descript_text)
    # llm에 context와 함께 제공해서, 시퀀셜한 자연어 문장으로 결과값 받기.
    llm_input = chart_descript_text + " the chart's html contents are: " +  chart_data
    llm_output = content_convert(prompt=chart_handler_prompt, data=llm_input)
    return llm_output



def figure_handler(figure_data: tuple):

    # figure_handler
    figure_handler_prompt = (
        "너는 도표 이미지를 설명하는 한국어 요약 전문가야.\n"
        "이미지의 대체 텍스트(markdown과 text)를 기반으로, 그 이미지가 무엇을 나타내는지 자연스럽게 설명해줘.\n"
        "수치가 포함된 경우에는 항목별로 나열하면서 간결하게 수치를 언급해주고, 전체적으로 어떤 내용을 담고 있는지 한국어로 시퀀셜하게 요약해줘.\n"
        "결과는 임베딩 벡터로 사용할 수 있도록 완전한 문장 구조를 가져야 하며, 단순 나열이 아닌 설명적인 형태로 구성되어야 해.\n"
        "형식 예시: 이 도표는 에너지 자원별 생산 비율을 보여준다. 석유는 34%, 석탄은 27%, 천연가스는 24% 등의 비중으로 구성된다."
    )

    # figure 원본 html 데이터.
    markdown = figure_data[0]["content"]["html"]

    # 방성식 정보위에 있는 문단의 데이터. (방정식에 대한 설명을 하고있을 확률 높음.) 해당 데이터는 None 이 들어올수있음. 그래도 오류가 나지않게 예외처리 해야함.
    figure_description = text_handler(figure_data[1])
    print(figure_description)
    llm_input = figure_description + " the figure's html contents are: " + markdown
    llm_ouput = content_convert(prompt=figure_handler_prompt, data=llm_input)
    return llm_ouput



In [ ]:
from typing import Callable, Dict

# === 카테고리 핸들러 매핑 ===
category_to_handler: Dict[str, Callable[[dict], str]] = {
    "equation": equation_handler,
    "chart": chart_handler,
    "figure": figure_handler,
    # 이하의 카테고리는 동일한 text_handler로 처리
}

# === 전체 파서 함수 ===
# data = datas['elements'][i]
# data_category = data['category']
# data_contents = data['contents']
def parse_data_by_category(data):
    data_category = data[0]['category'] if isinstance(data, tuple) else data['category']
    handler = category_to_handler.get(data_category, text_handler)
    return handler(data)

In [ ]:
def find_nearest_context_text(datas, current_index, max_search_num = 6):
    # 앞쪽 탐색 (우선순위)
    for offset in range(1, max_search_num):  # 최대 5칸까지 탐색
        idx = current_index - offset
        if idx >= 0:
            text = text_handler(datas[idx])
            if text.strip():
                return text

    # 뒤쪽 탐색 (앞쪽에 없을 경우)
    for offset in range(1, 6):
        idx = current_index + offset
        if idx < len(datas):
            text = text_handler(datas[idx])
            if text.strip():
                return text

    return ""  # 앞뒤 모두 실패 시


In [22]:
from collections import defaultdict

def group_by_page_with_handlers(datas):
    page_texts = defaultdict(list)

    for i, element in enumerate(datas):
        page = element.get("page", -1)
        category = element.get("category", "")

        if category in ["chart", "figure", "equation"]:
            context_text = find_nearest_context_text(datas, i)
            pair = (element, {"content": {"html": context_text}} if context_text else None)
            parsed_text = parse_data_by_category(pair)
        else:
            parsed_text = parse_data_by_category(element)

        page_texts[page].append(parsed_text)

    return [
        {"page": page, "content": "\n".join(texts)}
        for page, texts in sorted(page_texts.items())
    ]


# 사용 예시
grouped_pages = group_by_page_with_handlers(datas)

# 결과 출력 (디버깅용)
for page in grouped_pages:
    print(f"[Page {page['page']}]")
    print(page["content"])
    print("-" * 80)


Context Adherence Score
· AI 기술 중 근로자에 대한 경제적 가치(시간당 임금 증가율 기준) 측면에서 상위 5개 기술은
머신러닝(+40%), 텐서플로우(+38%), 딥러닝(+27%), 자연어처리(+19%), 데이터 과학(+17%) 순
[Page 1]
1. 정책/법제
2. 기업/산업
3. 기술/연구
4. 인력/교육
갈릴레오의 LLM 환각 지수 평가에서 GPT-4가 가장 우수
KEY Contents
주요 LLM의 환각 현상을 평가한 'LLM 환각 지수'에 따르면 GPT-4는 작업 유형과 관계없이
가장 우수한 성능을 보였으며 GPT-3.5도 거의 동등한 성능을 발휘
오픈소스 모델 중에서는 메타의 라마2가 RAG 없는 질문과 답변 및 긴 형식의 텍스트
생성에서 가장 우수한 성능을 발휘
주요 LLM 중 GPT-4가 가장 환각 현상 적고 GPT-3.5 터보도 비슷한 성능 기록
■ 머신러닝 데이터 관리 기업 갈릴레오(Galileo)가 2023년 11월 15일 주요 LLM의 환각 현상을 평가한
'LLM 환각 지수(LLM Hallucination Index)'를 발표
생성 AI의 환각 현상은 AI 시스템이 잘못된 정보를 생성하거나, 현실과 다른 부정확한 결과를 내놓는
현상으로, 기업의 AI 도입을 가로막는 주요 장애물이며, 환각 지수는 신뢰할 수 있는 생성 AI 구축을 위해
환각을 평가하고 측정하는 구조화된 접근방식을 제공
환각 지수는 △검색 증강 생성(Retrieval-Augmented Generation, RAG)*을 포함한 질문과 답변 △RAG
없는 질문과 답변 △긴 형식의 텍스트(보고서나 기사, 에세이) 생성의 3개 작업 유형에 대하여 환각을
기준으로 LLM의 순위를 평가
* 기존에 학습된 데이터가 아닌 외부 소스(데이터셋, 데이터베이스, 문서 등)에서 가져온 정보를 검색해 활용하는 기술
3개의 작업 유형 평가 전체에서 오픈AI의 GPT-4가 최고의 성능을 기록했으며, GPT-3.5 터보도
GPT-4와 거의 동등한 성능을 발휘
메타의 라마2(Lla

In [23]:
from langchain.schema import Document

def convert_grouped_pages_to_documents(grouped_pages):
    """
    grouped_pages: [{'page': 1, 'content': "..."}, ...]
    → [Document(page_content="...", metadata={"page": 1}), ...]
    """
    documents = []
    for page in grouped_pages:
        doc = Document(
            page_content=page['content'],
            metadata={"page": page['page']}
        )
        documents.append(doc)
    return documents


In [24]:
documents = convert_grouped_pages_to_documents(grouped_pages)
documents

[Document(metadata={'page': 1}, page_content="1. 정책/법제\n2. 기업/산업\n3. 기술/연구\n4. 인력/교육\n갈릴레오의 LLM 환각 지수 평가에서 GPT-4가 가장 우수\nKEY Contents\n주요 LLM의 환각 현상을 평가한 'LLM 환각 지수'에 따르면 GPT-4는 작업 유형과 관계없이\n가장 우수한 성능을 보였으며 GPT-3.5도 거의 동등한 성능을 발휘\n오픈소스 모델 중에서는 메타의 라마2가 RAG 없는 질문과 답변 및 긴 형식의 텍스트\n생성에서 가장 우수한 성능을 발휘\n주요 LLM 중 GPT-4가 가장 환각 현상 적고 GPT-3.5 터보도 비슷한 성능 기록\n■ 머신러닝 데이터 관리 기업 갈릴레오(Galileo)가 2023년 11월 15일 주요 LLM의 환각 현상을 평가한\n'LLM 환각 지수(LLM Hallucination Index)'를 발표\n생성 AI의 환각 현상은 AI 시스템이 잘못된 정보를 생성하거나, 현실과 다른 부정확한 결과를 내놓는\n현상으로, 기업의 AI 도입을 가로막는 주요 장애물이며, 환각 지수는 신뢰할 수 있는 생성 AI 구축을 위해\n환각을 평가하고 측정하는 구조화된 접근방식을 제공\n환각 지수는 △검색 증강 생성(Retrieval-Augmented Generation, RAG)*을 포함한 질문과 답변 △RAG\n없는 질문과 답변 △긴 형식의 텍스트(보고서나 기사, 에세이) 생성의 3개 작업 유형에 대하여 환각을\n기준으로 LLM의 순위를 평가\n* 기존에 학습된 데이터가 아닌 외부 소스(데이터셋, 데이터베이스, 문서 등)에서 가져온 정보를 검색해 활용하는 기술\n3개의 작업 유형 평가 전체에서 오픈AI의 GPT-4가 최고의 성능을 기록했으며, GPT-3.5 터보도\nGPT-4와 거의 동등한 성능을 발휘\n메타의 라마2(Llama-2-70b)는 RAG 없는 질문과 답변 유형에서 오픈소스 모델 가운데 가장 우수했고 긴\n형식의 텍스트 생성에서도 GPT-4에 준하는 성능을 기록

In [44]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.schema import Document

# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
# API 키 정보 로드
load_dotenv()


from langchain_teddynote import logging
# 프로젝트 이름을 입력합니다.
logging.langsmith("myRAG")

LangSmith 추적을 시작합니다.
[프로젝트명]
myRAG


In [26]:
# 단계 1: 문서 로드(Load Documents), 단계 2: 문서 분할(Split Documents)
documents

[Document(metadata={'page': 1}, page_content="1. 정책/법제\n2. 기업/산업\n3. 기술/연구\n4. 인력/교육\n갈릴레오의 LLM 환각 지수 평가에서 GPT-4가 가장 우수\nKEY Contents\n주요 LLM의 환각 현상을 평가한 'LLM 환각 지수'에 따르면 GPT-4는 작업 유형과 관계없이\n가장 우수한 성능을 보였으며 GPT-3.5도 거의 동등한 성능을 발휘\n오픈소스 모델 중에서는 메타의 라마2가 RAG 없는 질문과 답변 및 긴 형식의 텍스트\n생성에서 가장 우수한 성능을 발휘\n주요 LLM 중 GPT-4가 가장 환각 현상 적고 GPT-3.5 터보도 비슷한 성능 기록\n■ 머신러닝 데이터 관리 기업 갈릴레오(Galileo)가 2023년 11월 15일 주요 LLM의 환각 현상을 평가한\n'LLM 환각 지수(LLM Hallucination Index)'를 발표\n생성 AI의 환각 현상은 AI 시스템이 잘못된 정보를 생성하거나, 현실과 다른 부정확한 결과를 내놓는\n현상으로, 기업의 AI 도입을 가로막는 주요 장애물이며, 환각 지수는 신뢰할 수 있는 생성 AI 구축을 위해\n환각을 평가하고 측정하는 구조화된 접근방식을 제공\n환각 지수는 △검색 증강 생성(Retrieval-Augmented Generation, RAG)*을 포함한 질문과 답변 △RAG\n없는 질문과 답변 △긴 형식의 텍스트(보고서나 기사, 에세이) 생성의 3개 작업 유형에 대하여 환각을\n기준으로 LLM의 순위를 평가\n* 기존에 학습된 데이터가 아닌 외부 소스(데이터셋, 데이터베이스, 문서 등)에서 가져온 정보를 검색해 활용하는 기술\n3개의 작업 유형 평가 전체에서 오픈AI의 GPT-4가 최고의 성능을 기록했으며, GPT-3.5 터보도\nGPT-4와 거의 동등한 성능을 발휘\n메타의 라마2(Llama-2-70b)는 RAG 없는 질문과 답변 유형에서 오픈소스 모델 가운데 가장 우수했고 긴\n형식의 텍스트 생성에서도 GPT-4에 준하는 성능을 기록

In [58]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 2. 문서 분할 (chunking)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(documents)

 # 3. 임베딩 생성
embeddings = OpenAIEmbeddings()

# 4. 벡터스토어 생성
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

# 5. Retriever 생성
retriever = vectorstore.as_retriever()

# 6. LLM 생성
# llm = ChatOpenAI(model_name=model_name, temperature=0)
from openai import OpenAI # openai==1.52.2
client = OpenAI(
    api_key=api_key,
    base_url="https://api.upstage.ai/v1"
)


# Prompt 정의
prompt =  """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}
"""

# # question 정의
# question = "AI 기술 유형 평균 기술 대비 갖는 임금 프리미엄이 가장 높은 AI 기술은?"

# # print(chat_with_solar(messages))
# context = retriever.gt_relevant_documents("AI 기술 유형 평균 기술 대비 갖는 임금 프리미엄이 가장 높은 AI 기술은?")



In [60]:
question = "AI 기술 유형 평균 기술 대비 갖는 임금 프리미엄이 가장 높은 AI 기술은?"

In [59]:
top_k = 3  # 또는 5, 상황에 따라 조절
context_docs = retriever.get_relevant_documents(question)[:top_k]
context = "\n\n".join(doc.page_content for doc in context_docs)
context

"영국 옥스퍼드 인터넷 연구소, AI 기술자의 임금이 평균 21% 높아\nKEY Contents\n옥스퍼드 인터넷 연구소의 연구에 따르면 특정 기술의 경제적 가치는 다른 기술과 결합\n가능성이 높을수록 높게 평가됨\nAI의 확산은 기술의 경제적 가치에 크게 영향을 미치며, AI 기술을 가진 근로자는 평균 21%,\n최대 40% 높은 임금을 받을 수 있음\n。 AI 기술 중 머신러닝, 텐서플로우, 딥러닝의 임금 프리미엄이 높게 평가\n■ 옥스퍼드 인터넷 연구소(Oxford Internet Institute)가 2023년 10월 24일 962개 기술과 2만 5천\n명을 대상으로 한 연구에서 AI를 포함한 주요 기술의 경제적 가치를 분석한 결과를 발표\n연구에 따르면 한 기술의 경제적 가치는 근로자의 여타 역량과 얼마나 잘 결합하는지를 보여주는\n'상보성(complementarity)'에 따라 결정됨\n특정 기술은 다른 기술과 결합 가능성이 높을수록 경제적 가치가 높아지며, 일례로 데이터 분석과 같은\n\n이 도표는 AI 기술 중 근로자에 대한 경제적 가치(시간당 임금 증가율 기준) 측면에서 상위 5개 기술을 보여줍니다. 머신러닝 기술은 40%의 시간당 임금 증가율을 보이며, 텐서플로우 기술은 38%, 딥러닝 기술은 27%, 자연어처리(NLP) 기술은 20%, 데이터 과학 기술은 18%의 시간당 임금 증가율을 기록했습니다. 그 뒤를 이어 데이터 분석 기술은 15%, 파이썬 기술은 10%, C++ 기술은 8%, 자바 기술은 5%의 시간당 임금 증가율을 나타냅니다.\n☞ 출처 : Oxford Internet Institute, AI comes out on top: Oxford Study identifies the economic value of specific skills, 2023.10.24.\n18\n\n기술은 여타 고부가가치 기술과 결합할 수 있어 가치가 높지만, 사진 리터칭 같은 기술은 특정 기술과만\n결합할 수 있어 가치가 낮게 평가됨\n· 대부분 직업은 여러

In [52]:
# 2. prompt와 question을 조합해 user 메시지 생성
formatted_prompt = prompt.format(context=context, question=question)
formatted_prompt

"You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nAnswer in Korean.\n\n#Context:\n영국 옥스퍼드 인터넷 연구소, AI 기술자의 임금이 평균 21% 높아\nKEY Contents\n옥스퍼드 인터넷 연구소의 연구에 따르면 특정 기술의 경제적 가치는 다른 기술과 결합\n가능성이 높을수록 높게 평가됨\nAI의 확산은 기술의 경제적 가치에 크게 영향을 미치며, AI 기술을 가진 근로자는 평균 21%,\n최대 40% 높은 임금을 받을 수 있음\n。 AI 기술 중 머신러닝, 텐서플로우, 딥러닝의 임금 프리미엄이 높게 평가\n■ 옥스퍼드 인터넷 연구소(Oxford Internet Institute)가 2023년 10월 24일 962개 기술과 2만 5천\n명을 대상으로 한 연구에서 AI를 포함한 주요 기술의 경제적 가치를 분석한 결과를 발표\n연구에 따르면 한 기술의 경제적 가치는 근로자의 여타 역량과 얼마나 잘 결합하는지를 보여주는\n'상보성(complementarity)'에 따라 결정됨\n특정 기술은 다른 기술과 결합 가능성이 높을수록 경제적 가치가 높아지며, 일례로 데이터 분석과 같은\n\n이 도표는 AI 기술 중 근로자에 대한 경제적 가치(시간당 임금 증가율 기준) 측면에서 상위 5개 기술을 보여줍니다. 머신러닝 기술은 40%의 시간당 임금 증가율을 보이며, 텐서플로우 기술은 38%, 딥러닝 기술은 27%, 자연어처리(NLP) 기술은 20%, 데이터 과학 기술은 18%의 시간당 임금 증가율을 기록했습니다. 그 뒤를 이어 데이터 분석 기술은 15%, 파이썬 기술은 10%, C++ 기술은 8%, 자바 기술은 5%의 시간당 임금 증가율을 나타냅니다.\n☞ 출처 : Oxfor

In [74]:
def chat_with_solar(messages):
    response = client.chat.completions.create(
        model="solar-pro",
        messages=messages
    )
    return response.choices[0].message.content

In [ ]:
# 3. chat_with_solar에 넣을 messages 구성
messages = [{"role": "user", "content": formatted_prompt}]
print(chat_with_solar(messages))

AI 기술 유형 평균 기술 대비 갖는 임금 프리미엄이 가장 높은 AI 기술은 머신러닝입니다. 머신러닝 기술의 임금 프리미엄은 40%입니다. 



In [79]:
no_contet_question = "AI 기술 유형 평균 기술 대비 갖는 임금 프리미엄이 가장 높은 AI 기술은?"
no_contet_messages = [{"role": "user", "content": no_contet_question}]
print(chat_with_solar(no_contet_messages))

AI 기술 유형 중 평균 기술 대비 가장 높은 임금 프리미엄을 갖는 AI 기술은 자연어 처리(NLP)입니다. NLP 기술자는 평균 기술자에 비해 34%의 임금 프리미엄을 누리고 있습니다. 이는 다른 AI 기술들보다 상당히 높은 수준입니다. 예를 들어, 머신러닝 기술자는 26.3%의 프리미엄을, 컴퓨터 비전 기술자는 22.7%의 프리미엄을 누리고 있습니다.


# 비동기 처리

In [33]:
# 1. 비동기 요청 보내기 
import requests

api_key = "up_qAeV4wltdGnqdmxavmqmxXyRBpmcZ" # ex: up_xxxYYYzzzAAAbbbCCC
file_path = "data/sample_pdf.pdf" # ex: ./image.png

url = "https://api.upstage.ai/v1/document-digitization/async"
headers = {"Authorization": f"Bearer {api_key}"}
files={"document": open(file_path, "rb")}
data = {"model": "document-parse"}

response = requests.post(url, headers=headers, files=files, data = data)
request_id = response.json()["request_id"]

In [34]:
request_id

'5daee2eb-2fdf-486c-82fd-14245e054584'

In [35]:
url = f"https://api.upstage.ai/v1/document-digitization/requests/{request_id}"
headers = {"Authorization": f"Bearer {api_key}"}

response = requests.get(url, headers=headers)
result = response.json()
print(result)

{'id': '5daee2eb-2fdf-486c-82fd-14245e054584', 'status': 'submitted', 'model': 'document-parse', 'failure_message': '', 'total_pages': 0, 'completed_pages': 0, 'batches': [], 'requested_at': '2025-05-25T16:12:08Z', 'completed_at': ''}


In [36]:
result.keys()

dict_keys(['id', 'status', 'model', 'failure_message', 'total_pages', 'completed_pages', 'batches', 'requested_at', 'completed_at'])

In [37]:
result['completed_at']

''

In [38]:
result['status']

'submitted'

In [39]:
result["completed_pages"]

0

In [40]:
result["batches"]

[]

In [41]:
result

{'id': '5daee2eb-2fdf-486c-82fd-14245e054584',
 'status': 'submitted',
 'model': 'document-parse',
 'failure_message': '',
 'total_pages': 0,
 'completed_pages': 0,
 'batches': [],
 'requested_at': '2025-05-25T16:12:08Z',
 'completed_at': ''}

In [42]:
import requests

# 상태가 성공일 때만 처리
while result["status"] != "completed":
    download_url = result["batches"][0]["download_url"]
    response = requests.get(download_url)

    if response.status_code == 200:
        parsed_data = response.json()  # PDF 파싱된 페이지 리스트
        print("총 페이지 수:", len(parsed_data))
        print("첫 페이지 내용:", parsed_data[0])
    else:
        print("아직 처리 중이거나 실패함:", result["status"])
        parsed_data = None

# 필요 시 parsed_data 변수 사용
parsed_data

IndexError: list index out of range